<h1><center>Toronto neighborhood segmentation
</center></h1>


## 1. Importando e instalando librerías 


In [ ]:
!pip install beautifulsoup4
!pip install geocoder
!pip install pgeocode

In [176]:
import numpy as np # librería para manejar datos vectorizados
import pandas as pd # librería para el análisis de datos
import requests # librería para gestionar las peticiones
from bs4 import BeautifulSoup # librería para parsear documentos html
import pgeocode # librería para obtener localización geográfica
import folium # librería para graficar mapas
from sklearn.cluster import KMeans # importar k-means desde la fase de agrupación
# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

## 2. Adquiriendo los datos 

In [177]:
# Obteniendo la respuesta en formato html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response=requests.get(wikiurl)

In [178]:
#Convirtiendo los datos de html a un objeto beautifulsoup
soup = BeautifulSoup(response.text, 'html.parser')
#Buscamos la tabla objetivo en el objeto beautifulsoup
table = soup.find('table')
#obtenemos todas las celdas de la tabla y las guardamos en una variable
target = table.find_all('p')
#extraemos la información de las celdas
postal_codes = []
boroughs = []
neighborhoods = []

for cel in target:
    neighborhood = []
    postal_codes.append(cel.find('b').contents[0])
    if cel.find('span').find_all('a') == []:
        boroughs.append('Not assigned')
        neighborhoods.append('Not assigned')
    else:
        for item in cel.find('span').find_all('a'):
            neighborhood.append(item.contents[0])
        boroughs.append(neighborhood[0])
        neighborhood.pop(0)
        neighborhoods.append(neighborhood)
test = []
for item,x in zip(neighborhoods,boroughs):
    string = ' '
    if len(item) > 1 and isinstance(item,list):
        for x in item:
            string +=  ', ' + x
        string = string[3:]
        test.append(string)
    elif isinstance(item,list) and item != []:
        test.append(item[0])
    elif isinstance(item,list):
        test.append(x)
    else:
        test.append(item)

neighborhoods = test

In [179]:
#Creamos un dataframe con los datos obtenidos
data = {'PostalCode' : postal_codes, 
        'Borough': boroughs, 
        'Neighborhood': neighborhoods}
Toronto_neighborhoods = pd.DataFrame(data)
Toronto_neighborhoods = Toronto_neighborhoods[Toronto_neighborhoods['Borough'] != 'Not assigned'].reset_index(drop=True)
Toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


## 3. Adquiriendo latitud y longitud

In [180]:
# Inicializamos listas para almacenar las longitudes y latitudes de cada código postal
latitudes = []
longitudes = []
# Instanciamos un objeto pgeocode
query_object = pgeocode.Nominatim('CA')
# Obtenemos las longitudes y latitudes de cada código postal y las almacenamos en las listas previamente inicializadas
for code in Toronto_neighborhoods['PostalCode']:
    lat_lng_coords = query_object.query_postal_code(code)
    latitudes.append(lat_lng_coords.latitude)
    longitudes.append(lat_lng_coords.longitude)

In [181]:
# Agregamos las latitudes y longitudes al dataframe de vecindarios
Toronto_neighborhoods.insert(3,'Latitude',latitudes)
Toronto_neighborhoods.insert(4,'Longitude',longitudes)
# limpiamos los datos, comprobando si hay algún dato no valido y eliminado ese registro
Toronto_neighborhoods = Toronto_neighborhoods[Toronto_neighborhoods['Latitude'].notna()]
Toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
...,...,...,...,...,...
96,M8X,Etobicoke,"The Kingsway, Old Mill",43.6518,-79.5076
97,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
98,M7Y,Business reply mail,Business reply mail,43.7804,-79.2505
99,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu...",43.6325,-79.4939


## 4. Análisis 

In [182]:
# Obtenemos la latitud y la longitud de la ciudad de Toronto del primer registro de las listas de latitudes y longitudes

# crear un mapa de Toronto utilizando los valores de latitud y longitud

map_toronto = folium.Map(location=[latitudes[0],longitudes[0]], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(Toronto_neighborhoods['Latitude'], Toronto_neighborhoods['Longitude'], Toronto_neighborhoods['Borough'], Toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [183]:
Boroughs = Toronto_neighborhoods['Borough'].unique()
Boroughs

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'The Beaches', 'Dufferin',
       'Little Portugal', 'The Danforth', 'Brockton', 'India Bazaar',
       'Lawrence Park', 'Davisville', 'Forest Hill', 'High Park',
       'North Toronto', 'The Annex', 'Parkdale', 'Runnymede',
       'Moore Park', 'Summerhill', 'Business reply mail'], dtype=object)

In [240]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search"
        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3QhIOus484WX1aDIVjUu8YDVwC6yBaRDZUFuX35YWzS0="
        }
        data = {
            "radius": radius,
            "limit": 50,
            "ll" : str(lat) + ',' + str(lng)
        }
        # solicitud GET        
        response = requests.request("GET", url, headers=headers,data=data).json()
        results = response['results']        
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
        name, 
        lat, 
        lng, 
        v['name'],
        v['geocodes']['main']['latitude'], 
        v['geocodes']['main']['longitude'],  
        v['categories'][0]['name']) for v in results if len(v['categories']) != 0])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [241]:
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search"
        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3QhIOus484WX1aDIVjUu8YDVwC6yBaRDZUFuX35YWzS0="
        }
        data = {
            "radius": 500,
            "limit": 50,
            "ll" : str(43.7545) + ',' + str(-79.3300)
        }
        # solicitud GET        
        response = requests.request("GET", url, headers=headers,data=data).json()
        results = response['results']
        len(results[0]['categories'])

0

In [242]:
names, latitudes, longitudes =  list(Toronto_neighborhoods.loc[:,'Neighborhood']), list(Toronto_neighborhoods.loc[:,'Latitude']), list(Toronto_neighborhoods.loc[:,'Longitude']) 
venues = getNearbyVenues(names, latitudes, longitudes)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
North York
West Deane Park, Princess Gardens, Martin Grove, Islington
Rouge Hill, Port Union, Highland Creek
Don Mills, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Downtown Toronto
Caledonia-Fairbanks
Woburn
Leaside
Bay Street
Downtown Toronto
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview
Thorncliffe Park
Richmond, King
Dovercourt Village
Scarborough Village
Henry Farm
Northwood Park, York University
The Danforth
Harbourfront, Union Station, Toronto Islands
Trinity
Kennedy Park, Ionview, Birchmount Park
Bayview Village
Downsview, CFB Toronto
Riverdale
Toronto Dominion Centre, Design Exchange
Parkdale Village, Exhibition Place
Golden Mile, Clairlea, Oakridge
York Mills
Downsview
The Beac

In [243]:
len(venues['Neighborhood'].unique())

92

In [244]:
# codificación
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe en la primera posición
venues_onehot.insert(0,'Neighborhood',venues['Neighborhood']) 

venues_onehot.shape

(3865, 425)

### Agrupamos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría

In [245]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Addiction Treatment Center,Adult Store,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Amusement Park,...,Warehouse / Wholesale Store,Waste Management Service,Website Designer,Welding Service,Wholesaler,Wine Bar,Winery,Women's Store,Yoga Studio,Youth Organization
0,Agincourt,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.023256,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview",0.00000,0.046512,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bay Street,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bayview Village,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.058824,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,"Wexford, Maryvale",0.00000,0.041667,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
88,Willowdale,0.01087,0.010870,0.0,0.0,0.010870,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
89,Woburn,0.00000,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.035714,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
90,Woodbine Heights,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#### Escribimos una función para ordenar los sitios en orden descendente.

In [246]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada barrio.

In [247]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Business and Professional Services,Real Estate Agency,Clothing Store,Industrial Equipment Supplier,Painter,General Contractor,Sporting Goods Retail,Convenience Store,Furniture and Home Store,Recycling Facility
1,"Alderwood, Long Branch",Business and Professional Services,Home Improvement Service,Photography Service,Convenience Store,"Cafes, Coffee, and Tea Houses",Construction,Recreation Center,Automotive Repair Shop,Travel Agency,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview",Business and Professional Services,Gift Store,Accounting and Bookkeeping Service,Flower Store,Carpenter,Taco Restaurant,Furniture and Home Store,Burger Joint,Computer Repair Service,Loans Agency
3,Bay Street,Café,Japanese Restaurant,Diner,Coffee Shop,Asian Restaurant,Bakery,Sushi Restaurant,Bank,Swimming Pool,Chinese Restaurant
4,Bayview Village,Business and Professional Services,Interior Designer,Property Management Office,Pest Control Service,Radio Station,General Contractor,Architecture Firm,Dry Cleaner,Jewelry Store,Website Designer
...,...,...,...,...,...,...,...,...,...,...,...
87,"Wexford, Maryvale",Business and Professional Services,Costume Store,Doors and Windows Contractor,Cosmetics Store,Hair Salon,Attorney / Law Office,Restaurant,Carpet and Flooring Contractor,Caterer,Pest Control Service
88,Willowdale,"Cafes, Coffee, and Tea Houses",Convenience Store,Professional Cleaning Service,Cosmetics Store,Café,Health and Medicine,Sushi Restaurant,Shopping Mall,Automotive Repair Shop,Restaurant
89,Woburn,Business and Professional Services,Accounting and Bookkeeping Service,Retail,Drugstore,Restaurant,Cosmetics Store,General Contractor,Retirement Home,Grocery Store / Supermarket,Clothing Store
90,Woodbine Heights,Roofer,Garden Center,Park,Convenience Store,Photographer,Media Agency,Carpet and Flooring Contractor,Retail,Garden,Furniture and Home Store


In [248]:
neighborhoods_venues_sorted.shape

(92, 11)

# 4. Agrupando barrios


Ejecutemos k-means para agrupar los barrios en 5 agrupaciones. 

In [249]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

C:\Users\User\AppData\Local\Temp\ipykernel_10024\2084648263.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  toronto_grouped_clustering = venues_grouped.drop('Neighborhood', 1)


array([1, 1, 1, 3, 1, 3, 1, 1, 3, 1])

In [250]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_neighborhoods

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # revisar las ultimas columnas

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,1,Business and Professional Services,Accounting and Bookkeeping Service,Park,Community and Government,Financial Service,Swimming Pool,General Contractor,Carpenter,Engineer,Health Food Store
1,M4A,North York,Victoria Village,43.7276,-79.3148,4,Car Dealership,General Contractor,Check Cashing Service,Automotive Repair Shop,"Cafes, Coffee, and Tea Houses",Banking and Finance,Tailor,Bridal Store,Furniture and Home Store,Stadium
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,3,Automotive Repair Shop,Antique Store,Music Venue,Art Gallery,Coffee Shop,Asian Restaurant,Hair Salon,Café,Sushi Restaurant,Gym and Studio
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,3,Clothing Store,Bakery,Restaurant,Toy / Game Store,Department Store,Coffee Shop,Cosmetics Store,Shoe Store,Fast Food Restaurant,Retail
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889,3,Café,Diner,Italian Restaurant,Fast Food Restaurant,Clothing Store,"Cafes, Coffee, and Tea Houses",Tattoo Parlor,Bubble Tea Shop,Thai Restaurant,Sports Bar


In [257]:
# crear mapa
map_clusters = folium.Map(location=[latitudes[0], longitudes[0]], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examinar Agrupaciones

#### Agrupación 1

In [260]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Scarborough,0,Organization,Other Great Outdoors,Organic Grocery,Optometrist,Ophthalmologist,Oil Change Service,Office Supply Store,Nutritionist,Noodle Restaurant,Non-Profit Organization


#### Agrupación 2

In [261]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Business and Professional Services,Accounting and Bookkeeping Service,Park,Community and Government,Financial Service,Swimming Pool,General Contractor,Carpenter,Engineer,Health Food Store
5,Etobicoke,1,Hair Salon,Grocery Store / Supermarket,Drugstore,General Contractor,Park,Bank,Baseball Field,Media Agency,Telecommunication Service,Landscaper and Gardener
6,Scarborough,1,Print Store,Audiovisual Service,Business and Professional Services,Home Service,Food Distribution Center,Automotive Repair Shop,Construction Supplies Store,Arts and Entertainment,Import and Export Service,Movie Theater
7,North York,1,Business and Professional Services,General Contractor,Elementary School,Import and Export Service,Construction Supplies Store,Music Store,Swimming Pool,Event Service,Telecommunication Service,Loans Agency
8,East York,1,Public and Social Service,Hair Salon,Pet Grooming Service,Flower Store,Drugstore,Clothing Store,Print Store,Convenience Store,Construction,Caterer
11,Etobicoke,1,Bakery,Pizzeria,Electronics Store,Financial Planner,Construction Supplies Store,Construction,Media Agency,Sports and Recreation,Real Estate Service,Filipino Restaurant
13,North York,1,Park,Dive Bar,Bridge,Media Agency,Garden Center,Event Service,Loans Agency,Bank,River,Landmarks and Outdoors
14,East York,1,Roofer,Garden Center,Park,Convenience Store,Photographer,Media Agency,Carpet and Flooring Contractor,Retail,Garden,Furniture and Home Store
16,York,1,Playground,Park,Sporting Goods Retail,Grocery Store / Supermarket,Media Agency,Computer Repair Service,Landmarks and Outdoors,Gift Store,Hardware Store,Hair Salon
17,Etobicoke,1,Bank,Drugstore,Park,Management Consultant,Liquor Store,ATM,Massage Clinic,Party Supply Store,Barbershop,Grocery Store / Supermarket


#### Agrupación 3

In [264]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,2,Park,Harbor / Marina,ATM,New American Restaurant,Organic Grocery,Optometrist,Ophthalmologist,Oil Change Service,Office Supply Store,Nutritionist


#### Agrupación 4

In [265]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Automotive Repair Shop,Antique Store,Music Venue,Art Gallery,Coffee Shop,Asian Restaurant,Hair Salon,Café,Sushi Restaurant,Gym and Studio
3,North York,3,Clothing Store,Bakery,Restaurant,Toy / Game Store,Department Store,Coffee Shop,Cosmetics Store,Shoe Store,Fast Food Restaurant,Retail
4,Queen's Park,3,Café,Diner,Italian Restaurant,Fast Food Restaurant,Clothing Store,"Cafes, Coffee, and Tea Houses",Tattoo Parlor,Bubble Tea Shop,Thai Restaurant,Sports Bar
9,Downtown Toronto,3,Fast Food Restaurant,Department Store,Clothing Store,Chinese Restaurant,Coffee Shop,Beer Bar,Bookstore,"Cafes, Coffee, and Tea Houses",Japanese Restaurant,Movie Theater
10,North York,3,Hair Salon,Bakery,Pizzeria,Caterer,Financial Planner,Nail Salon,Bar,Business and Professional Services,"Heating, Ventilating and Air Conditioning Cont...",Lounge
15,Downtown Toronto,3,Café,Coffee Shop,Pizzeria,Diner,Asian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Sports Bar,Camera Store,Burger Joint
19,The Beaches,3,Café,Pizzeria,Bicycle Store,Park,Burger Joint,Business and Professional Services,Pet Grooming Service,Pub,Grocery Store / Supermarket,Dance Studio
20,Downtown Toronto,3,Restaurant,Café,Coffee Shop,Pub,"Cafes, Coffee, and Tea Houses",American Restaurant,Grocery Store / Supermarket,Diner,Liquor Store,Beer Bar
23,East York,3,Bank,Restaurant,Health Food Store,Sushi Restaurant,Pizzeria,Fast Food Restaurant,Deli,Senior Citizen Service,Retail,Collections Service
24,Downtown Toronto,3,Café,Japanese Restaurant,Diner,Coffee Shop,Asian Restaurant,Bakery,Sushi Restaurant,Bank,Swimming Pool,Chinese Restaurant


#### Agrupación 5

In [267]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Car Dealership,General Contractor,Check Cashing Service,Automotive Repair Shop,"Cafes, Coffee, and Tea Houses",Banking and Finance,Tailor,Bridal Store,Furniture and Home Store,Stadium
12,Scarborough,4,Automotive Repair Shop,Hair Salon,General Contractor,Computer Repair Service,Used Car Dealership,Swim School,Carpet and Flooring Contractor,Fabric / Textiles Store,Retirement Home,Bar
50,North York,4,Automotive Repair Shop,General Contractor,Car Parts and Accessories,Business and Professional Services,Retail,Gym and Studio,Metals Supplier,Carpet and Flooring Contractor,Transmissions Shop,Food and Beverage Retail
63,Scarborough,4,Automotive Repair Shop,Tire Repair Shop,Clothing Store,Monument,Construction Supplies Store,Professional Cleaning Service,Carpenter,Martial Arts Dojo,Retail,Party Supply Store
92,Etobicoke,4,Car Dealership,Automotive Repair Shop,Used Car Dealership,Business and Professional Services,Automotive Retail,Computer Repair Service,Waste Management Service,Health and Medicine,Locksmith,Hair Salon
100,Etobicoke,4,Automotive Repair Shop,Fruit and Vegetable Store,Waste Management Service,Burrito Restaurant,Café,Spa,Car Parts and Accessories,Cheese Store,Fast Food Restaurant,Construction Supplies Store
